# CLIP 模型直接微调（对比学习）

目标：使用电商数据中的图片与文本对，直接在 CLIP 上进行对比学习微调，并保存微调后的权重供后续检索使用。
- 使用项目现有 `DataLoader` 逐批加载图片与查询文本，构造图文对。
- 采用 CLIP 的对比损失（图像-文本相互对齐），支持 AMP 降内存与加速。
- 提供资源友好的默认参数，适配 32GB RAM / 16GB VRAM。


In [1]:
import os
import math
import json
from typing import List

import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader as TorchDataLoader
from torchvision import transforms
from PIL import Image

from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

from transformers import CLIPProcessor, CLIPModel, BertTokenizer
from transformers import get_cosine_schedule_with_warmup

# 环境与缓存设置
cache_dir = "/mnt/d/HuggingFaceModels/"
os.environ['TORCH_HOME'] = cache_dir
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['all_proxy'] = 'socks5://127.0.0.1:7890'
os.environ["WANDB_DISABLED"] = "true"
os.environ['CURL_CA_BUNDLE'] = ""
os.environ['HF_ENDPOINT'] = "https://hf-mirror.com"

import sys
sys.path.append(os.path.abspath(os.path.join('.', 'Multimodal_Retrieval', 'plan1_1')))
from data_loader import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

save_dir = '/mnt/d/forCoding_data/Tianchi_MUGE/trained_models/weights'
os.makedirs(save_dir, exist_ok=True)
clip_save_path = os.path.join(save_dir, 'ft_clip_vit_base_patch32.pth')


/home/xiuminke/miniconda3/envs/ml12/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/xiuminke/miniconda3/envs/ml12/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
import time
while True:
    if os.path.exists("ft_rsn50_brt.finishflag"):
        break
    time.sleep(5)

In [3]:
class CLIPPairDataset(Dataset):
    def __init__(self, items, queries_texts, image_transform=None):
        self.items = items
        self.texts = queries_texts
        self.tf = image_transform
        self.n = min(len(self.items), len(self.texts))
    def __len__(self):
        return self.n
    def __getitem__(self, idx):
        it = self.items[idx]
        img = it.get('image', None)
        text = self.texts[idx]
        if img is None:
            img = Image.new('RGB', (224, 224), color=(128,128,128))
        if self.tf is not None:
            img = self.tf(img)
        return img, text

def build_clip_processor_model():
    processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32', cache_dir=cache_dir, local_files_only=True)
    model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32', cache_dir=cache_dir, local_files_only=True).to(device)
    return processor, model

def collate_fn(batch):
    # 此 collate 不再使用，训练循环里会定义基于 CLIPProcessor 的版本
    imgs = [b[0] for b in batch]
    texts = [b[1] for b in batch]
    return {'pixel_values': torch.stack(imgs, dim=0), 'texts': texts}


In [4]:
def train_clip(loader: DataLoader, device, epochs=1, img_batch_size=800, inner_bs=64, lr=5e-5, max_batches=2):
    # 使用 CLIPProcessor 进行文本编码，移除 BERT tokenizer
    processor, model = build_clip_processor_model()
    scaler = GradScaler(enabled=(device.type=='cuda'))
    optim = AdamW(model.parameters(), lr=lr)
    # 估算总训练步数用于LR调度（近似即可，有助稳定下降）
    total_batches_est = max(1, max_batches or 1)
    steps_per_batch_est = max(1, img_batch_size // inner_bs)
    total_steps_est = total_batches_est * steps_per_batch_est
    warmup_steps = max(1, int(0.2 * total_steps_est))
    scheduler = get_cosine_schedule_with_warmup(optim, num_warmup_steps=warmup_steps, num_training_steps=total_steps_est)
    image_tf = None

    model.train()
    # 仅加载一次训练查询，并在各批次循环中重复利用
    train_df = loader.load_queries(split='train')
    train_texts = [t for t in train_df.get('query_text', []) if isinstance(t, str)]
    if len(train_texts) == 0:
        return model
    text_offset = 0

    outer = loader.load_images_batch(split='train', batch_size=img_batch_size, max_batches=max_batches)
    for batch_idx, img_batch in enumerate(outer):
        # 基于 item_ids 构造当前批次的真实图文正样本对
        # 统一键为字符串，避免类型不一致导致无法匹配
        img_map = {str(it['img_id']): it['image'] for it in img_batch if it.get('image', None) is not None}
        paired_items = []
        paired_texts = []
        if 'item_ids' in train_df.columns:
            for _, row in train_df.iterrows():
                q = row.get('query_text', None)
                ids = row.get('item_ids', [])
                # 兼容字符串形式的列表，如 '[123, 456]' 或 '123,456'
                if isinstance(ids, str):
                    parsed = None
                    try:
                        parsed = json.loads(ids)
                    except Exception:
                        try:
                            parsed = [s.strip() for s in ids.split(',') if s.strip()]
                        except Exception:
                            parsed = None
                    ids = parsed if isinstance(parsed, list) else []
                if not q or not isinstance(ids, list) or not ids:
                    continue
                chosen_img = None
                for iid in ids:
                    sid = str(iid)
                    if sid in img_map:
                        chosen_img = img_map[sid]
                        break
                if chosen_img is not None:
                    paired_items.append({'image': chosen_img})
                    paired_texts.append(q)
        if len(paired_items) == 0:
            continue
        ds = CLIPPairDataset(paired_items, paired_texts, image_tf)
        def collate_clip(batch):
            imgs = [b[0] for b in batch]
            texts = [b[1] for b in batch]
            inputs = processor(text=texts, images=imgs, return_tensors='pt', padding=True, truncation=True)
            return {
                'pixel_values': inputs['pixel_values'],
                'input_ids': inputs['input_ids'],
                'attention_mask': inputs.get('attention_mask', None)
            }
        dl = TorchDataLoader(ds, batch_size=inner_bs, shuffle=True, num_workers=2, pin_memory=(device.type=='cuda'), collate_fn=collate_clip)
        running_loss = 0.0
        steps = 0
        # pbar = tqdm(dl, desc=f'Train batch {batch_idx+1}', leave=False)
        for i, batch in enumerate(dl, 1):
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
            with autocast(enabled=(device.type=='cuda')):
                outputs = model(**batch)
                # 直接使用 logits 计算对比损失
                logits_per_image = outputs.logits_per_image
                logits_per_text = outputs.logits_per_text
                # 跳过过小微批（对比学习至少需要 >1 的批量用于负样本）
                bs = logits_per_image.size(0)
                if bs < 2:
                    continue
                targets = torch.arange(bs, device=device)
                # 标签平滑可缓解偶发的损失尖峰
                ce = nn.CrossEntropyLoss(label_smoothing=0.1)
                loss_i = ce(logits_per_image, targets)
                loss_t = ce(logits_per_text, targets)
                loss = (loss_i + loss_t) / 2.0
            optim.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optim)
            scaler.update()
            scheduler.step()
            # 累计与显示损失
            running_loss += float(loss.detach().item())
            steps += 1
            avg = running_loss / max(1, steps)
            # pbar.set_postfix({'loss': f'{loss.item():.4f}', 'avg': f'{avg:.4f}'})
        current_lr = optim.param_groups[0]['lr']
        print(f'Batch {batch_idx+1}: avg loss={running_loss/max(1, steps):.4f} | lr={current_lr:.6f}')
        if device.type == 'cuda':
            torch.cuda.empty_cache()
    return model


In [5]:
loader = DataLoader()
clip_model = train_clip(loader, device=device, img_batch_size=2000, max_batches=100, inner_bs=16, epochs=3, lr=1e-4)
torch.save(clip_model.state_dict(), clip_save_path)
print(f'CLIP fine-tuned and saved to: {clip_save_path}')


2025-11-09 21:53:09,376 - INFO - 初始化数据加载器，数据目录: /mnt/d/forCoding_data/Tianchi_MUGE/originalData/Multimodal_Retrieval
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/tmp/ipykernel_225180/2793667927.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(device.type=='cuda'))
2025-11-09 21:53:14,020 - INFO - 加载train查询数据: /mnt/d/forCoding_data/Tianchi_MUGE/originalData/Multimodal_Retrieval/MR_train_queries.jsonl
加载train查询数据: 248786it [00:00, 263029.12it/s]
2025-11-09 21:53:15,027 - INFO - 成功加载train查询数据，共248786条
2025-11-09 21:53:15,047 - INFO - 批量加载train图片数据: /mnt/d/forCoding_data/Tianchi_MUGE/originalData/Multi

Batch 1: avg loss=2.4462 | lr=0.000010


实际加载train图片数据:   3%|█▌                                                  | 4000/129380 [00:44<1:07:32, 30.94it/s]

Batch 2: avg loss=1.9444 | lr=0.000020


实际加载train图片数据:   5%|██▌                                                   | 6000/129380 [01:06<57:54, 35.51it/s]

...

Batch 63: avg loss=1.5576 | lr=0.000018


实际加载train图片数据:  99%|███████████████████████████████████████████████████▍| 128000/129380 [22:43<00:35, 38.52it/s]

Batch 64: avg loss=1.5802 | lr=0.000021


实际加载train图片数据: 100%|████████████████████████████████████████████████████| 129380/129380 [22:43<00:00, 94.88it/s]


Batch 65: avg loss=1.5692 | lr=0.000023
CLIP fine-tuned and saved to: /mnt/d/forCoding_data/Tianchi_MUGE/trained_models/weights/ft_clip_vit_base_patch32.pth


## 验证评估：Recall@1/5/10 与 MeanRecall
- 基于验证集构建图像索引
- 对每条查询计算相似度并统计召回（优先使用 FAISS；不可用则 Torch 回退）


In [6]:
import importlib, numpy as np
HAS_FAISS = False
faiss = None
try:
    spec = importlib.util.find_spec('faiss')
    if spec is not None:
        faiss = importlib.import_module('faiss')
        HAS_FAISS = True
except BaseException as e:
    print(f'Faiss unavailable: {e.__class__.__name__}')
    HAS_FAISS = False

# 加载验证集查询
valid_df = loader.load_queries(split='valid')
valid_queries = []
if 'item_ids' in valid_df.columns:
    for _, row in valid_df.iterrows():
        q = row.get('query_text', None)
        ids = [str(i) for i in row.get('item_ids', [])] if isinstance(row.get('item_ids', []), list) else []
        if q and ids:
            valid_queries.append((q, ids))
print(f'Usable valid queries: {len(valid_queries)}')

# 构建验证集图像字典（可选限制最大样本数）
valid_imgs_max_samples = None  # 可改为例如 30000 以降低内存
valid_imgs = loader.create_img_id_to_image_dict(split='valid', max_samples=valid_imgs_max_samples)

# 使用 CLIP 计算图像特征并构建索引
processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32', cache_dir=cache_dir, local_files_only=True)
clip_model.eval()
def l2_normalize(x):
    return torch.nn.functional.normalize(x, p=2, dim=-1)

image_index = {}
all_ids = list(valid_imgs.keys())
batch_size = 64
for s in tqdm(range(0, len(all_ids), batch_size), desc='Build image index'):
    batch_ids = all_ids[s:s+batch_size]
    batch_imgs = [valid_imgs[i] for i in batch_ids if valid_imgs[i] is not None]
    valid_batch_ids = [i for i in batch_ids if valid_imgs[i] is not None]
    if not batch_imgs:
        continue
    inputs = processor(images=batch_imgs, return_tensors='pt')
    pixel_values = inputs['pixel_values'].to(device)
    with torch.no_grad():
        feats = clip_model.get_image_features(pixel_values)
        feats = l2_normalize(feats).detach().cpu()
    for j, img_id in enumerate(valid_batch_ids):
        image_index[img_id] = feats[j]

all_image_ids = list(image_index.keys())
all_image_feats = torch.stack([image_index[i] for i in all_image_ids]) if all_image_ids else torch.empty((0, clip_model.visual_projection.out_features if hasattr(clip_model, 'visual_projection') else 512))
faiss_index = None
if HAS_FAISS and all_image_feats.size(0) > 0:
    d = all_image_feats.size(1)
    faiss_index = faiss.IndexFlatIP(d)
    faiss_index.add(all_image_feats.numpy().astype('float32'))

all_image_feats = all_image_feats.to(device)

def compute_recall_at_k(k_values, queries):
    recalls = {k: 0 for k in k_values}
    total = 0
    for q_text, gt_ids in tqdm(queries, desc='Evaluate'):
        if all_image_feats.size(0) == 0:
            continue
        inputs = processor(text=[q_text], return_tensors='pt', padding=True)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs.get('attention_mask', None)
        if attention_mask is not None:
            attention_mask = attention_mask.to(device)
        with torch.no_grad():
            q_feat = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
            q_feat = l2_normalize(q_feat)
        if faiss_index is not None:
            q_np = q_feat.detach().cpu().numpy().astype('float32')
            _, I = faiss_index.search(q_np, max(k_values))
            top_idx = I[0].tolist()
            top_ids = [all_image_ids[i] for i in top_idx]
        else:
            sims = (q_feat @ all_image_feats.t()).squeeze(0)
            _, top_idx = torch.topk(sims, k=max(k_values))
            top_ids = [all_image_ids[i] for i in top_idx.tolist()]
        total += 1
        for k in k_values:
            if any(g in set(top_ids[:k]) for g in gt_ids):
                recalls[k] += 1
    return {k: (recalls[k] / max(1, total)) for k in k_values}, total

k_values = [1, 5, 10]
recall_scores, N = compute_recall_at_k(k_values, valid_queries)
mean_recall = sum(recall_scores.values()) / len(k_values)
print(f"Recall@1={recall_scores[1]:.4f}, Recall@5={recall_scores[5]:.4f}, Recall@10={recall_scores[10]:.4f}, MeanRecall={mean_recall:.4f} (N={N})")


2025-11-09 22:16:26,338 - INFO - Loading faiss with AVX2 support.
2025-11-09 22:16:26,408 - INFO - Successfully loaded faiss with AVX2 support.
2025-11-09 22:16:26,413 - INFO - 加载valid查询数据: /mnt/d/forCoding_data/Tianchi_MUGE/originalData/Multimodal_Retrieval/MR_valid_queries.jsonl
加载valid查询数据: 5008it [00:00, 349031.66it/s]
2025-11-09 22:16:26,433 - INFO - 成功加载valid查询数据，共5008条


Usable valid queries: 5008


2025-11-09 22:16:26,511 - INFO - 批量加载valid图片数据: /mnt/d/forCoding_data/Tianchi_MUGE/originalData/Multimodal_Retrieval/MR_valid_imgs.tsv
实际加载valid图片数据: 100%|████████████████████████████████████████████████████| 29806/29806 [00:14<00:00, 2053.30it/s]
2025-11-09 22:16:43,723 - INFO - 成功创建valid图片映射字典，共29806张图片
Evaluate: 100%|████████████████████████████████████████████████████████████████████| 5008/5008 [00:43<00:00, 115.74it/s]

Recall@1=0.0132, Recall@5=0.0537, Recall@10=0.0931, MeanRecall=0.0533 (N=5008)


In [7]:
with open("ft_clip.finishflag", "w") as f:
    f.write("finish")

import IPython
def kill_current_kernel():
    '''杀死当前的kernel释放内存空间。'''
    IPython.Application.instance().kernel.do_shutdown(True)
kill_current_kernel()